<a href="https://colab.research.google.com/github/freehtet/NLP/blob/main/m508a_big_data_analytics_ws0924__wai_yann_htet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Statement


My company, local coffee shop, assigned me to work on the customers' reviews from yelp. Nowadays, it is easier to access to receive customers feedback and all the feedbacks matter as it can boost the company sales or reduce the company sales. Previously, there are only handful of review, so our team can work on reviews manually. As internet user growth, it is a bit out of hand, I have been assigned to create a machine learning model which can provide whether customers reviews are positive, negative or neutral. This will help company to quickly see what area of the shop need to be improved and eventually increase the sales.

Data has been collected from the Kaggle. (https://www.kaggle.com/datasets/sripaadsrinivasan/yelp-coffee-reviews/data).



#High level system design

In general, I have created 2 model, 1. is using transformer and 2. is using neural network.
By comparing both performance and recommend the best model to fit with the NLP task.
As of process, in general high level,

Step 1. Data collection, data has been downloaded used from Kaggle as mentioned above.

Step 2. Data processing and feature engineering, even though transformer model can tokenization, encoding and paddings, I also used Spacy library to do tokenization for neural network model. Additionally, as data is imbalanced, I have sample data form each rating to have balance dataset. Finally, label data has been changed and categorized from numerical ratings to the Positive, Neutral and Negative.

Step 3. Model training and evaluation: Data have been spliced into Train and test by 20%. Models will be trained on the provided data ( reviews and ratings) by using transformer and neural network. The evaluation will be done with accuracy by using test dataset in the end.

Overall, these steps are very important in implementation of NLP model. Starting from Step 2, if the data is not properly cleaned and pre-process, the output result will be impacted. On the other hand, it can help model to performance better. As Step 3, is the very important part of the process as model required to be trained with to understand the area of doing analyzing and evaluation of the efficiency of the model which can be based to make the decision whether the model should be implemented or require different approach.

#Importing Libraries

In [ ]:
!pip install evaluate
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
import spacy
import sklearn.metrics
# import sklearn.preprocessing
import sklearn.model_selection
from sklearn.preprocessing import LabelEncoder
import datasets
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import evaluate
import transformers
from prettytable import PrettyTable

#Loading Dataset

In [ ]:
raw = pd.read_csv('raw_yelp_review_data.csv')

In [ ]:
raw.head()

,coffee_shop_name,full_review_text,star_rating
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5.0 star rating
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4.0 star rating
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4.0 star rating
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2.0 star rating
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4.0 star rating


In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7616 entries, 0 to 7615
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   coffee_shop_name  7616 non-null   object
 1   full_review_text  7616 non-null   object
 2   star_rating       7616 non-null   object
dtypes: object(3)
memory usage: 178.6+ KB


In [ ]:
raw['star_rating'].value_counts()

,count
star_rating,
5.0 star rating,3780
4.0 star rating,2360
3.0 star rating,738
2.0 star rating,460
1.0 star rating,278


Data Processing

Removing interger and text combinaition of rating in dataset.

In [ ]:
raw['ratings'] = raw['star_rating'].apply(lambda x: int(x[:2].strip()))

Removing date from the text.

In [ ]:
raw['full_review_text'] = raw['full_review_text'].apply(lambda x: x[11:])

Loading Spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

Applying Spacy into review text to clean text as part of text pre-processing for the neural network model.

In [ ]:
raw['clean text'] = raw['full_review_text'].apply(lambda x: nlp(x))

In [ ]:
raw['text'] = raw['clean text'].apply(lambda x: ' '.join([token.text for token in x]))

Creating new DF with new column name

In [ ]:
df = raw[['full_review_text','text', 'ratings']]
df.columns = ['text','c_text', 'label']
df.head()

,text,c_text,label
0,1 check-in Love love loved the atmosphere! Ev...,1 check - in Love love loved the atmosphere ...,5
1,"Listed in Date Night: Austin, Ambiance in Aust...","Listed in Date Night : Austin , Ambiance in Au...",4
2,1 check-in Listed in Brunch Spots I loved the...,1 check - in Listed in Brunch Spots I loved ...,4
3,Very cool decor! Good drinks Nice seating Ho...,Very cool decor ! Good drinks Nice seating ...,2
4,1 check-in They are located within the Northcr...,1 check - in They are located within the North...,4


Checking dataset in terms of label distribution

In [ ]:
df["label"].value_counts()

,count
label,
5,3780
4,2360
3,738
2,460
1,278


Spliting the dataset based on rating to create the well balance dataset from sample

In [ ]:
df0 = df[df['label'] == 1]
df1 = df[df['label'] == 2]
df2 = df[df['label'] == 3]
df3 = df[df['label'] == 4]
df4 = df[df['label'] == 5]

Createing new DF with 100 sample from each rating except from rating 2 with 200 sample. Total 600 sample taken out.

In [ ]:
df_final = pd.concat([df0.sample(100), df1.sample(100), df2.sample(200), df3.sample(100), df4.sample(100)])

In [ ]:
df_final['label'].value_counts()

,count
label,
3,200
1,100
2,100
4,100
5,100


Randomizing the DF in order spread out the data.

In [ ]:
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_final.head()

,text,c_text,label
0,1 check-in Wifi is fast! Mocha's cinnamony an...,1 check - in Wifi is fast ! Mocha 's cinnamo...,2
1,check-in Listed in Pretending to Hook 'Em The...,check - in Listed in Pretending to Hook ' Em...,4
2,Came here one evening as I was in the mood for...,Came here one evening as I was in the mood for...,5
3,Did you know that humans are very perceptive c...,Did you know that humans are very perceptive c...,1
4,think someone is likely being asked to write ...,think someone is likely being asked to write...,2


Creating the function to change rating into textual Positive, Negative and Netural

In [ ]:
def label_change(num):
  if num < 3:
    return 'Negative'
  elif num == 3:
    return 'Neutral'
  else:
    return 'Postivie'

Applying the function into label column

In [ ]:
df_final['label'] = df_final['label'].apply(lambda x: label_change(x))

Encoding the label

In [ ]:
le = LabelEncoder()
df_final['label'] = le.fit_transform(df_final['label'])

Train Test split for the Transformer model.

In [ ]:
df_train, df_test = sklearn.model_selection.train_test_split(df_final, test_size=0.2, random_state=42)

Transformer model used is DistilBert and auto tokenizer from the model. Then, use padding to have same lenght for text.

In [ ]:
model_name = 'distilbert-base-uncased'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

Create a function to truncate the text with max length 128 to make sure all text have same length.

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

Encoding training and testing data by using of funcation created above.

In [ ]:
df_train_encoded = datasets.Dataset.from_pandas(df_train).map(preprocess_function, batched=True)
df_test_encoded = datasets.Dataset.from_pandas(df_test).map(preprocess_function, batched=True)

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Creating the evaluation metric to complute the performance of the model.

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Loading pre-trained model.

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=df_final['label'].nunique())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating the arguments for training

In [ ]:
training_args = transformers.TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    report_to="none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Seting up trainer with above created arguments for training data and setting up the evaluation metric.

In [ ]:
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train_encoded,
    eval_dataset=df_test_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-171-965dd5b7b96b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Training the model and saving the model.

In [ ]:
trainer.train()
trainer.save_model("./results/" + model_name)

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.029361,0.550000
2,No log,0.897136,0.641667
3,No log,0.776269,0.641667
4,No log,0.775498,0.625000
5,No log,0.764406,0.683333
6,No log,0.798131,0.666667
7,No log,0.835302,0.691667
8,No log,0.892594,0.708333
9,No log,0.919443,0.700000
10,No log,0.925001,0.691667


Evaluating the training result

In [ ]:
trainer.evaluate()

{'eval_loss': 0.9250008463859558,
 'eval_accuracy': 0.6916666666666667,
 'eval_runtime': 25.2537,
 'eval_samples_per_second': 4.752,
 'eval_steps_per_second': 0.317,
 'epoch': 10.0}

#Neural network

Creating the neural network to work on same process as above to compare the 2 different model.

Splitting the training and testing set.

In [ ]:
X = df_final['c_text']
y = df_final['label']

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

Creating the max words and lenght

In [ ]:
max_words = 40000
max_len = 200

Tokenizing the text

Tokenizing the words with max words and fit the tokenizer based on X_train.

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

Creating the sequence of number based on tokenizer instead of text.

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

Padding process have been done on sequence in order to have the same length of data.

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

Creating the sequntial layers for deep learning neural network with LSTM , along side with dropout, batch normalization.

In [ ]:
model = models.Sequential([
    layers.Embedding(max_words, 128),
    layers.SpatialDropout1D(0.2),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.BatchNormalization(),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="softmax")
])

Creating adam optimizer with learning rate 0.002.

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.002)

Compiling the model.

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Creating early stopping to prevent overfitting and time comsuming.

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

Training process start and stroing the data in histroy.

In [ ]:
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 537ms/step - accuracy: 0.3440 - loss: 0.0000e+00 - val_accuracy: 0.3438 - val_loss: 0.0000e+00
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 401ms/step - accuracy: 0.3440 - loss: 0.0000e+00 - val_accuracy: 0.3438 - val_loss: 0.0000e+00
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 578ms/step - accuracy: 0.3440 - loss: 0.0000e+00 - val_accuracy: 0.3438 - val_loss: 0.0000e+00
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 402ms/step - accuracy: 0.3440 - loss: 0.0000e+00 - val_accuracy: 0.3438 - val_loss: 0.0000e+00


Tesing on testset to get result in unseem data.

In [ ]:
y_pred = model.predict(X_test_pad)

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 320ms/step


In [ ]:
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred.round())

In [ ]:
accuracy *100

35.833333333333336

Creating Table to compare the results.

In [ ]:
table = PrettyTable()

In [ ]:
table.field_names = ['Model', 'Accuracy']
table.add_row(['Transformer model', 65])
table.add_row(['Neural network model', 35])

In [ ]:
display('The result of models')
print(table)

'The result of models'

+----------------------+----------+
|        Model         | Accuracy |
+----------------------+----------+
|  Transformer model   |    65    |
| Neural network model |    35    |
+----------------------+----------+


#Final Discussion

Since I have created 2 models, there are difference requirement to properly train models. Firslty, as of the result of the model, using Transformer model is very effictive in term of evaluation result. However, using Transformer model would be time comsuming. Therefore, only sample of dataset can be training in order to save time and with the limited resource. On the other hand, using the Neural network can be a bit quite faster in term of model training with the Transformer model. However, the result of model is not as good as using transformer model.
Overall, based on the comparison of the result between Transformer model and Neural network, I would recommended to use Transformer model even though it take time and resources can be sigfinicantly required compared to neural network model.
